In [1]:
import pickle 
import pandas as pd 
from sklearn.feature_extraction import DictVectorizer 
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5003")
mlflow.set_experiment("green-taxi-duration")

2025/06/18 12:32:46 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1750206766729, experiment_id='1', last_update_time=1750206766729, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
# Read dataframe 
def read_dataframe(filename: str):
    "Wrtie a function to read dataframe"
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    "Write a function to prepare dictionaries"
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient = 'records')
    return dicts 

In [5]:
df_train = read_dataframe('../../00-Dataset/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../../00-Dataset/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values 
y_val = df_val[target].values 

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [12]:
with mlflow.start_run(run_name="XGBoost Regressor"):
    # Log tags for developer and model
    mlflow.set_tag("developer", "Dario")
    mlflow.set_tag("model_name", "xgboost")

    # XGBoost parameters
    params = dict(
        max_depth=6,
        learning_rate=0.1,
        n_estimators=100,
        objective='reg:squarederror',
        random_state=42
    )
    mlflow.log_params(params)

    dv = DictVectorizer()

    pipeline = make_pipeline(
        dv,
        xgb.XGBRegressor(**params)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = root_mean_squared_error(y_pred, y_val)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    # Log model with MLflow
    mlflow.sklearn.log_model(pipeline, artifact_path="model")

    # Save and log DictVectorizer separately
    with open('dict_vectorizer.bin', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact('dict_vectorizer.bin')

{'max_depth': 6, 'learning_rate': 0.1, 'n_estimators': 100, 'objective': 'reg:squarederror', 'random_state': 42} 6.714536758841806


/Users/dazieldang/.local/share/virtualenvs/code-axWtE436/lib/python3.9/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/dazieldang/.local/share/virtualenvs/code-axWtE436/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [9]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = 'http://127.0.0.1:5003/'
RUN_ID = '83ec3045b32240a8a853fcc5907f39d6/'
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)